In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

c:\Users\banfi\anaconda3\envs\pytorch-env\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Load Dataset

In [3]:
df = pd.read_csv("task2_data.txt")
train_labels = pd.read_csv("task2_train_label.txt", header=None, sep="\t", names=["order", "label"])
valid_labels = pd.read_csv("task2_valid_label.txt", header=None, sep="\t", names=["order", "label"])
test_query = pd.read_csv("task2_test_query.txt",header=None, names=["order"])

In [4]:
df

,order,product,customer,color,size,group
0,391395,20019,106251,557,21,15
1,391395,20114,106251,396,21,15
2,112116,45195,135056,243,24,23
3,112116,6117,135056,338,24,23
4,739883,622,233090,586,25,23
...,...,...,...,...,...,...
2666257,576096,9045,15446,462,24,8
2666258,314874,20257,217058,437,21,15
2666259,314874,40360,217058,65,25,15
2666260,314874,46311,217058,460,25,8


In [6]:
test_query["label"] = -1

In [9]:
df

,order,product,customer,color,size,group
0,391395,20019,106251,557,21,15
1,391395,20114,106251,396,21,15
2,112116,45195,135056,243,24,23
3,112116,6117,135056,338,24,23
4,739883,622,233090,586,25,23
...,...,...,...,...,...,...
2666257,576096,9045,15446,462,24,8
2666258,314874,20257,217058,437,21,15
2666259,314874,40360,217058,65,25,15
2666260,314874,46311,217058,460,25,8


In [11]:
train_labels.sort_values(by="order", inplace=False)

,order,label
428605,0,0
264235,0,0
372583,0,0
472603,0,1
726572,0,0
...,...,...
505983,58414,0
353385,58414,1
624211,58414,1
105302,58414,1


In [14]:
train_df = df.merge(train_labels["label"], left_index=True, right_index=True)
valid_df = df.merge(valid_labels["label"], left_index=True, right_index=True)
test_df = df.merge(test_query["label"], left_index=True, right_index=True)

In [15]:
train_df

,order,product,customer,color,size,group,label
0,391395,20019,106251,557,21,15,0
0,391395,20019,106251,557,21,15,0
0,391395,20019,106251,557,21,15,1
1,391395,20114,106251,396,21,15,1
1,391395,20114,106251,396,21,15,0
...,...,...,...,...,...,...,...
849182,551062,28554,150579,472,15,23,1
849182,551062,28554,150579,472,15,23,0
849183,551062,57168,150579,472,15,7,1
849183,551062,57168,150579,472,15,7,0


In [16]:
labels = pd.concat([train_labels, valid_labels, test_query])

total_length = len(train_labels) + len(valid_labels) + len(test_query)

train_masks = np.concatenate([np.ones(len(train_labels)), np.zeros(total_length - len(train_labels))])
valid_masks = np.concatenate([np.zeros(len(train_labels)), np.ones(len(valid_labels)), np.zeros(len(test_query))])
test_masks = np.concatenate([np.zeros(total_length - len(test_query)), np.ones(len(test_query))])

masks = pd.DataFrame({"order": labels["order"],"train_masks": train_masks, "valid_masks": valid_masks, "test_masks": test_masks})
masks

,order,train_masks,valid_masks,test_masks
428605,0,1.0,0.0,0.0
332050,0,1.0,0.0,0.0
358090,0,1.0,0.0,0.0
196086,0,1.0,0.0,0.0
677671,0,1.0,0.0,0.0
...,...,...,...,...
399934,115842\t31870,0.0,0.0,1.0
399935,504278\t53317,0.0,0.0,1.0
399936,144537\t33864,0.0,0.0,1.0
399937,622382\t22404,0.0,0.0,1.0


In [49]:
train_df.head(3)

,order,product,customer,color,size,group,label
0,298957,54654,192219,169,10,11,0
1,298957,57127,192219,611,10,11,0
2,570617,23677,322757,43,24,19,1


In [9]:
df = df.merge(labels, on="order")
df.head(3)

,order,product,customer,color,size,group,label
0,298957,54654,192219,169,10,11,0
1,298957,57127,192219,611,10,11,0
2,570617,23677,322757,43,24,19,1


# Create Customer Edgelist

In [51]:
def get_customer_edgelist(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns the customer edgelist with the following edge attributes: customer, order, n_products, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing customer edgelist.
    """
    # Group by customer
    customer_groups = df.groupby(['order', 'customer', 'label']).size().reset_index().rename(columns={0: 'n_products'})
    return customer_groups[['customer', 'order', 'n_products', 'label']].sort_values(by='customer')

def save_customer_edgelist(df: pd.DataFrame, filename: str) -> None:
    """
    Saves the customer edgelist to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer edgelist.
        filename (str): Filename to save the customer edgelist to.
    """
    output = get_customer_edgelist(df)
    output.to_csv(filename, sep=',', index=False)

In [71]:
# save_customer_edgelist(train_df, 'train_customer_edgelist.csv')
customer_edge_list = get_customer_edgelist(df)
customer_edge_list = customer_edge_list.merge(masks, on="order")
customer_edge_list.to_csv("customer_edgelist.csv", index=False)

# Create Order Edgelist

In [7]:
def get_order_edgelist(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns the order edgelist with the following edge attributes: order, product, n_customers, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing order edgelist.
    """
    # Group by order
    order_groups = df.groupby(['order', 'product', 'label','color','size']).size().reset_index().rename(columns={0: 'n_customers'})
    return order_groups[['order', 'product', 'label','color','size']].sort_values(by='order')

def save_order_edgelist(df: pd.DataFrame, filename: str) -> None:
    """
    Saves the order edgelist to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing order edgelist.
        filename (str): Filename to save the order edgelist to.
    """
    output = get_order_edgelist(df)
    output.to_csv(filename, sep=',', index=False)

In [10]:
# save_order_edgelist(train_df, 'train_order_edgelist.csv')
save_order_edgelist(df, 'order_edgelist.csv')

# Create Customer Node Attributes
Possibly find some way to find the average size of product ordered, but also linked to product type.

In [70]:
def get_customer_node_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns the customer node attributes with the following attributes: customer, n_orders, n_products, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing customer node attributes.
    """
    # Group by customer
    customer_groups = df.groupby(['customer', 'order', 'label']).size().reset_index().rename(columns={0: 'n_products'})
    customer_groups['n_not_returned'] = customer_groups['label'].apply(lambda x: 1 if x == 0 else 0)
    customer_groups['n_partial_returned'] = customer_groups['label'].apply(lambda x: 1 if x == 1 else 0)
    customer_groups['n_fully_returned'] = customer_groups['label'].apply(lambda x: 1 if x == 2 else 0)
    customer_groups = customer_groups.groupby('customer').agg({'order': 'count', 'n_not_returned': 'sum', 'n_partial_returned': 'sum', 'n_fully_returned': 'sum', 'n_products': 'sum'}).reset_index().rename(columns={'order': 'n_orders'})
    return customer_groups

def save_customer_node_attributes(df: pd.DataFrame, filename: str) -> None:
    """
    Saves the customer node attributes to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer node attributes.
        filename (str): Filename to save the customer node attributes to.
    """
    output = get_customer_node_attributes(df)
    output.to_csv(filename, sep=',', index=False)

In [67]:
save_customer_node_attributes(df, 'customer_node_attributes.csv')

# Create Order Node Attributes

In [12]:
def get_order_node_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns the order node attributes with the following attributes: customer, n_orders, n_products, label
    
    Args:
        df (pandas.DataFrame): DataFrame containing customer order information.
    Returns:
        pandas.DataFrame: DataFrame containing customer node attributes.
    """
    customer_groups = df.groupby(['order', 'label']).size().reset_index().rename(columns={0: 'n_products'})
    return customer_groups

def save_order_node_attributes(df: pd.DataFrame, filename: str) -> None:
    """
    Saves the order node attributes to a file.
    
    Args:
        df (pandas.DataFrame): DataFrame containing order node attributes.
        filename (str): Filename to save the order node attributes to.
    """
    output = get_order_node_attributes(df)
    output.to_csv(filename, sep=',', index=False)

In [68]:
# save_order_node_attributes(train_df, 'train_order_node_attributes.csv')
save_order_node_attributes(df, 'order_node_attributes.csv')

# Create Product Node Attributes

In [9]:
#todo: Finish this
def get_product_node_attributes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the node attributes for each product in the given DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the product data.

    Returns:
    pd.DataFrame: A DataFrame with the calculated node attributes for each product.
    """
    # Function implementation goes here
    pass

    # Group by customer
    product_stats = df.groupby('product').agg(
        n_orders=('product', 'count'),
        n_returned=('label', lambda x: (x==1).sum() + (x==2).sum())
    ).reset_index(drop=False)

    return product_stats

def save_product_node_attributes(df: pd.DataFrame, filename: str) -> pd.DataFrame:
    """
    Save the product node attributes to a CSV file.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the product node attributes.
    - filename (str): The name of the CSV file to save the attributes to.

    Returns:
    - pd.DataFrame: The DataFrame containing the product node attributes.

    """
    output = get_product_node_attributes(df)
    output.to_csv(filename, index=False)



In [10]:
# Returns calculated only on training data
save_product_node_attributes(train_df, "data/product_node_attributes.csv")